## 수정사항
#### 2021.06.06
* T_ej는 마지막 조인트 각도에 영향을 받음, 하지만 Q는 unkown이므로 특정하는게 불가능, T_ee로 전부 새로 학습할 것.

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
DATA_PATH = os.path.join(PROJ_DIR, "data")
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

In [3]:
ROBOT_TYPE_NAME="indy7"
ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)
ROBOT_DATA_ROOT_FAILS = ROBOT_DATA_ROOT+"-failmore"
# ROBOT_DATA_ROOT = LAT_DATA_PATH
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME+"-failmore")
ARM_FOLDER = ARM10_FOLDER

In [4]:
dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN = dataset_list[:10]
DATASET_TEST = dataset_list[10:15]
print(DATASET_TRAIN)
print(DATASET_TEST)

dataset_list_fails = sorted([folder for folder in os.listdir(ROBOT_DATA_ROOT_FAILS) if not folder.startswith(".")])
DATASET_TRAIN_FAILS = dataset_list_fails[:10]
DATASET_TEST_FAILS = dataset_list_fails[10:15]
print(DATASET_TRAIN_FAILS)
print(DATASET_TEST_FAILS)

['20210214-232708', '20210215-041031', '20210215-085110', '20210215-133753', '20210215-184319', '20210216-005455', '20210216-054418', '20210216-104554', '20210216-152114', '20210216-201729']
['20210217-010926', '20210217-063641', '20210217-113319', '20210217-162106', '20210217-205606']
['20210607-155241', '20210607-161407', '20210607-163215', '20210607-165616', '20210607-171344', '20210607-173927', '20210607-175629', '20210607-181751', '20210607-183958', '20210607-190513']
['20210607-192529', '20210607-194820', '20210607-200835', '20210607-202843', '20210607-205048']


In [5]:
GRASP_SHAPE = (20,20,20)
ARM_SHAPE = (20,20,20)
RH_MASK_SIZE = 512
RH_MASK_STEP = 64

data_pairs_train_succ = []
for dataset in DATASET_TRAIN:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_succ.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("train set: {}".format(len(data_pairs_train_succ)))        
        
        
data_pairs_test_succ = []
for dataset in DATASET_TEST:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_succ.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("test set: {}".format(len(data_pairs_test_succ)))        

data_pairs_train_fails = []
for dataset in DATASET_TRAIN_FAILS:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILS, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_fails.append((os.path.join(ROBOT_DATA_ROOT_FAILS, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILS, dataset, ARM_FOLDER, file)))
print("train fail set: {}".format(len(data_pairs_train_fails)))        
        
        
data_pairs_test_fails = []
for dataset in DATASET_TEST_FAILS:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILS, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_fails.append((os.path.join(ROBOT_DATA_ROOT_FAILS, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILS, dataset, ARM_FOLDER, file)))
print("test fail set: {}".format(len(data_pairs_test_fails)))        

train set: 10022
test set: 5002
train fail set: 10044
test fail set: 5016


In [6]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def div_r_gaussian(r_val):
    return gaussian(r_val, np.arange(0.1,1.2, 0.05),0.1)

def div_h_gaussian(h_val):
    return gaussian(h_val, np.arange(-0.5,1.1, 0.05),0.1)

def load_data(data_pair):
    grasp_data = load_pickle(data_pair[0])
    arm_data = load_pickle(data_pair[1])
    grasp_obj_idx = grasp_data[b'obj']
    grasp_tar_idx = grasp_data[b'tar']
    grasp_tool_idx = grasp_data[b'tool']
    arm_tar_idx = arm_data[b'tar']
    Tee = grasp_data[b'T_end_effector']
    Tej = grasp_data[b'T_end_joint']
    Tref_base = grasp_data[b'Tref_base']
    reach_lb = grasp_data[b'reach']
    retrieve_lb = grasp_data[b'retrieve']
    r, th, h = cart2cyl(*Tee[:3,3])
#     r_ej, th_ej, h_ej = cart2cyl(*Tej[:3,3])  # not exact value, no use
    r_mask = div_r_gaussian(r)
    h_mask = div_h_gaussian(h)
    rh_mask = np.concatenate([r_mask, h_mask])
#     rh_mask = np.array([r, h, r_ej, h_ej])
    # r_ej_list.append(r_ej)
    # h_ej_list.append(h_ej)
    # reach_lb_list.append(reach_lb)
#     Tref = SE3(Rot_axis(3, th), Tee[:3,3])
    grasp_tool_img = np.zeros(GRASP_SHAPE)
    grasp_tar_img = np.zeros(GRASP_SHAPE)
    grasp_obj_img = np.zeros(GRASP_SHAPE)
    grasp_tool_img[np.unravel_index(grasp_tool_idx, shape=GRASP_SHAPE)] = 1
    grasp_tar_img[np.unravel_index(grasp_tar_idx, shape=GRASP_SHAPE)] = 1
    grasp_obj_img[np.unravel_index(grasp_obj_idx, shape=GRASP_SHAPE)] = 1
    arm_img = np.zeros(ARM_SHAPE+(1,))
    arm_img[np.unravel_index(arm_tar_idx, shape=ARM_SHAPE)] = 1
    grasp_img = np.stack([grasp_tool_img, grasp_obj_img, grasp_tar_img], axis=-1)
#     grasp_img = np.stack([grasp_tool_img, np.logical_or(grasp_obj_img, grasp_tar_img)], axis=-1)
#     grasp_img = np.stack([grasp_tool_img, grasp_tar_img], axis=-1)
    label = np.array([reach_lb, retrieve_lb])
    return grasp_img, arm_img, rh_mask, label

In [7]:
data_pairs_train = []
for data_pairs in data_pairs_train_succ:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if False: #all(label):
        data_pairs_train.append(data_pairs)
N_succ = len(data_pairs_train)
print("train set prev: {}".format(N_succ))        


for data_pairs in data_pairs_train_fails:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if True: #not all(label):
        data_pairs_train.append(data_pairs)
N_train = len(data_pairs_train)
N_fail = N_train - N_succ
print("train set new: {}".format(N_fail))
print("train set all: {}".format(N_train))

data_pairs_test = []
for data_pairs in data_pairs_test_succ:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if False: #all(label):
        data_pairs_test.append(data_pairs)
N_succ = len(data_pairs_test)
print("test set prev: {}".format(N_succ))        

succount = 0
for data_pairs in data_pairs_test_fails:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if not all(label):
        if np.random.uniform() < 0.2:
            data_pairs_test.append(data_pairs)
    else:
        succount += 1
        data_pairs_test.append(data_pairs)
        data_pairs_test.append(data_pairs)
        data_pairs_test.append(data_pairs)
        data_pairs_test.append(data_pairs)
        data_pairs_test.append(data_pairs)
        data_pairs_test.append(data_pairs)
        data_pairs_test.append(data_pairs)
N_test = len(data_pairs_test)
N_fail = N_test - N_succ
print("test set new: {}".format(N_fail))
print("test set all: {}".format(N_test))
print('success in test set: {}'.format(succount))

train set prev: 0
train set new: 10044
train set all: 10044
test set prev: 0
test set new: 2300
test set all: 2300
success in test set: 194


## Train

In [8]:
from pkg.planning.filtering.lattice_model.lattice_model import *

# Create an instance of the model
model = ResNetModelTP()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [9]:
current_time = get_now()
logpath = os.path.join(ROBOT_MODEL_ROOT, current_time)
try_mkdir(logpath)
train_log_dir = os.path.join(logpath, 'train')
test_log_dir = os.path.join(logpath, 'test')
model_log_dir = os.path.join(logpath, 'model_{}/')
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
shutil.copy(os.path.join(PROJ_DIR,'src', 'pkg','planning','filtering','lattice_model','lattice_model.py' ), logpath)
print(f'Log path: {logpath}')

Log path: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210607-224401


In [10]:
EPOCHS_S = 0
EPOCHS_E = 15
BATCH_SIZE = 16
LOG_STEP = 100
N_train = len(data_pairs_train)
N_test = len(data_pairs_test)
random.shuffle(data_pairs_test)

for epoch in range(EPOCHS_S, EPOCHS_E):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    random.shuffle(data_pairs_train)
    i_step = 0
    data_batch, label_batch = [], []
    for data_pair in data_pairs_train:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        data_batch.append([grasp_img, arm_img, rh_mask])
        label_batch.append(label)
        if i_step%BATCH_SIZE==0:
            grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
            arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
            rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
            label_batch = np.array(label_batch, dtype=np.int)
            train_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
            data_batch, label_batch = [], []
        if i_step%LOG_STEP==0:
            print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    i_step = 0
    data_batch, label_batch = [], []
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        data_batch.append([grasp_img, arm_img, rh_mask])
        label_batch.append(label)
        if i_step%BATCH_SIZE==0:
            grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
            arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
            rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
            label_batch = np.array(label_batch, dtype=np.int)
            test_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
            data_batch, label_batch = [], []
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
            
    model.save(model_log_dir.format(epoch + 1))

    print("")
    print("=================================================================")
    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )
    print("=================================================================")
    print("")

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210607-224401/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210607-224401/model_1/assets



Epoch 1, Loss: 0.22860348224639893, Accuracy: 92.95753479003906, Test Loss: 0.9430172443389893, Test Accuracy: 52.931034088134766



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210607-224401/model_2/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210607-224401/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210607-224401/model_14/assets



Epoch 14, Loss: 0.07456223666667938, Accuracy: 96.7354507446289, Test Loss: 0.7118741869926453, Test Accuracy: 67.5



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210607-224401/model_15/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210607-224401/model_15/assets



Epoch 15, Loss: 0.07111723721027374, Accuracy: 96.96471405029297, Test Loss: 0.6207495927810669, Test Accuracy: 75.77586364746094



 ```
  4 0.67 69.52
  8 1.10 73.69
12 0.53 79.50
```


## New train 2021-06
* **[IMPORTANT] 이전에 Tej 사용 틀린 점 보완 했음** 

### Panda
* 20210606-080139 : first balanced train trial, 96.6% 달성 / 이전 데이터와 collision boundary shape 차이일 수 있음
* 20210606-082302 : 이전 데이터와 전부 섞어서 collision boundary shape 차이 영향 확인 - 87% 가량, 영향 확실, 재수집 필요
* 20210606-191455 : 새로 모았어도 분리 학습 하니 시작부터 99%, 문제 있음
* 20210606-194148 : 새로 모은거 분리 학습, 테스트만 순수 기존 데이터로 하니 59%, 문제 확실
* 20210606-201525 : 새 데이터만 학습, 96%? 하지만 fail bias 돼 있음.
* 20210606-204309 : 새 데이터만 학습, test set은 bias 제거함 ep8 89% 수준

### Indy7
* 20210607-224401 : 새 데이터만 학습, test set은 bias 제거함 ep12 79.5% 수준

## Load & test

20210606-204309 : 새 데이터만 학습, test set은 bias 제거함 ep8 89% 수준

In [8]:
print(sorted(os.listdir(ROBOT_MODEL_ROOT)))

['20210607-224401']


In [9]:
# last_model = '20210606-204309' # panda
last_model = '20210607-224401' # indy
logpath = os.path.join(ROBOT_MODEL_ROOT, last_model)

model_epoch_list = []
acc_epoch_list = []
loss_epoch_list = []
epochlist = sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:]))
print(epochlist)

['model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6', 'model_7', 'model_8', 'model_9', 'model_10', 'model_11', 'model_12', 'model_13', 'model_14', 'model_15']


In [10]:
last_save = 'model_12'
# last_save = 'model_1'
model_log_dir = os.path.join(logpath, last_save)

import tensorflow as tf
model = tf.keras.models.load_model(model_log_dir)

@tf.function
def inference(images):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    return predictions

loss_object = tf.keras.losses.BinaryCrossentropy()

@tf.function
def calc_loss(labels, predictions):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    return loss_object(labels, predictions)

grasp_img, arm_img, rh_mask, label = load_data(data_pairs_test[0])
res = inference([np.array([grasp_img]), np.array([arm_img]), np.array([rh_mask])])

#### old data

In [11]:
data_pairs_test = []
for data_pairs in data_pairs_test_succ:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if True: #all(label):
        data_pairs_test.append(data_pairs)
N_succ = len(data_pairs_test)
print("test set prev: {}".format(N_succ))        

for data_pairs in data_pairs_test_fails:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if False: # not all(label):
        data_pairs_test.append(data_pairs)
N_test = len(data_pairs_test)
N_fail = N_test - N_succ
print("test set new: {}".format(N_fail))
print("test set all: {}".format(N_test))

test set prev: 5002
test set new: 0
test set all: 5002


In [12]:
BATCH_SIZE = 1
LOG_STEP = 100
N_test = len(data_pairs_test)
gtimer = GlobalTimer.instance()
gtimer.reset()


i_step = 0
res_list = []
label_list = []
loss_list= []
grasp_img_batch = []
arm_img_batch = []
rh_mask_batch = []
label_batch = [] 
for data_pair in data_pairs_test:
    i_step += 1
    grasp_img, arm_img, rh_mask, label = load_data(data_pair)
    grasp_img_batch.append(grasp_img)
    arm_img_batch.append(arm_img)
    rh_mask_batch.append(rh_mask)
    label_batch.append(label)
    if len(grasp_img_batch)==BATCH_SIZE:
        grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
        with gtimer.block("inference"):
            res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
        loss = calc_loss(label_batch, res)
        res_list = res_list + list(res.numpy()>0.5)
        label_list = label_list + label_batch
        loss_list.append(loss.numpy())
        grasp_img_batch = []
        arm_img_batch = []
        rh_mask_batch = []
        label_batch = [] 
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

res_list = np.array(res_list)[:5000,1]
label_list = np.array(label_list)[:5000,1]
loss_list = np.array(loss_list)[:5000]

acc = np.mean(np.equal(res_list, label_list)) * 100
mean_loss = np.mean(loss_list)

print("")
print("=================================================================")
print(
    f'Test Loss: {mean_loss} \n'
    f'Test Accuracy: {acc} \n'
    f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
    f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
    f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
    f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
    f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
    f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
    f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
    f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
    f'{round(np.mean(res_list==label_list)*100,2)}\n'
)
print("=================================================================")
print("")
print(gtimer)
model_epoch_list.append(last_save)
acc_epoch_list.append(acc)
loss_epoch_list.append(mean_loss)

test step - 5000/5002        
Test Loss: 0.7262680530548096 
Test Accuracy: 71.82 
TP / FN / ACC: 1205, 1118, 51.87
FP / TN / ACC: 291, 2386, 89.13
PACC / NACC / TACC: 80.55, 68.09, 71.82


inference: 	130659.0 ms/5002 = 26.121 ms (12.001/49.606)



#### new data

In [13]:
data_pairs_test = []
for data_pairs in data_pairs_test_succ:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if False: #all(label):
        data_pairs_test.append(data_pairs)
N_succ = len(data_pairs_test)
print("test set prev: {}".format(N_succ))        

for data_pairs in data_pairs_test_fails:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if True: # not all(label):
        data_pairs_test.append(data_pairs)
N_test = len(data_pairs_test)
N_fail = N_test - N_succ
print("test set new: {}".format(N_fail))
print("test set all: {}".format(N_test))

test set prev: 0
test set new: 5016
test set all: 5016


In [14]:
BATCH_SIZE = 1
LOG_STEP = 100
N_test = len(data_pairs_test)
gtimer = GlobalTimer.instance()
gtimer.reset()


i_step = 0
res_list = []
label_list = []
loss_list= []
grasp_img_batch = []
arm_img_batch = []
rh_mask_batch = []
label_batch = [] 
for data_pair in data_pairs_test:
    i_step += 1
    grasp_img, arm_img, rh_mask, label = load_data(data_pair)
    grasp_img_batch.append(grasp_img)
    arm_img_batch.append(arm_img)
    rh_mask_batch.append(rh_mask)
    label_batch.append(label)
    if len(grasp_img_batch)==BATCH_SIZE:
        grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
        with gtimer.block("inference"):
            res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
        loss = calc_loss(label_batch, res)
        res_list = res_list + list(res.numpy()>0.5)
        label_list = label_list + label_batch
        loss_list.append(loss.numpy())
        grasp_img_batch = []
        arm_img_batch = []
        rh_mask_batch = []
        label_batch = [] 
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

res_list = np.array(res_list)[:5000,1]
label_list = np.array(label_list)[:5000,1]
loss_list = np.array(loss_list)[:5000]

acc = np.mean(np.equal(res_list, label_list)) * 100
mean_loss = np.mean(loss_list)

print("")
print("=================================================================")
print(
    f'Test Loss: {mean_loss} \n'
    f'Test Accuracy: {acc} \n'
    f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
    f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
    f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
    f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
    f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
    f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
    f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
    f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
    f'{round(np.mean(res_list==label_list)*100,2)}\n'
)
print("=================================================================")
print("")
print(gtimer)
model_epoch_list.append(last_save)
acc_epoch_list.append(acc)
loss_epoch_list.append(mean_loss)

test step - 5000/5016        
Test Loss: 0.0794977992773056 
Test Accuracy: 96.06 
TP / FN / ACC: 129, 64, 66.84
FP / TN / ACC: 133, 4674, 97.23
PACC / NACC / TACC: 49.24, 98.65, 96.06


inference: 	131483.0 ms/5016 = 26.213 ms (11.646/52.214)



#### mixed data

In [15]:
data_pairs_test = []
for data_pairs in data_pairs_test_succ:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if all(label):
        data_pairs_test.append(data_pairs)
N_succ = len(data_pairs_test)
print("test set prev: {}".format(N_succ))        

for data_pairs in data_pairs_test_fails:
    grasp_img, arm_img, rh_mask, label = load_data(data_pairs)
    if not all(label):
        data_pairs_test.append(data_pairs)
        if len(data_pairs_test) > 2*N_succ:
            break
N_test = len(data_pairs_test)
N_fail = N_test - N_succ
print("test set new: {}".format(N_fail))
print("test set all: {}".format(N_test))

test set prev: 2323
test set new: 2324
test set all: 4647


In [16]:
BATCH_SIZE = 1
LOG_STEP = 100
N_test = len(data_pairs_test)
gtimer = GlobalTimer.instance()
gtimer.reset()


i_step = 0
res_list = []
label_list = []
loss_list= []
grasp_img_batch = []
arm_img_batch = []
rh_mask_batch = []
label_batch = [] 
for data_pair in data_pairs_test:
    i_step += 1
    grasp_img, arm_img, rh_mask, label = load_data(data_pair)
    grasp_img_batch.append(grasp_img)
    arm_img_batch.append(arm_img)
    rh_mask_batch.append(rh_mask)
    label_batch.append(label)
    if len(grasp_img_batch)==BATCH_SIZE:
        grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
        with gtimer.block("inference"):
            res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
        loss = calc_loss(label_batch, res)
        res_list = res_list + list(res.numpy()>0.5)
        label_list = label_list + label_batch
        loss_list.append(loss.numpy())
        grasp_img_batch = []
        arm_img_batch = []
        rh_mask_batch = []
        label_batch = [] 
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

res_list = np.array(res_list)[:5000,1]
label_list = np.array(label_list)[:5000,1]
loss_list = np.array(loss_list)[:5000]

acc = np.mean(np.equal(res_list, label_list)) * 100
mean_loss = np.mean(loss_list)

print("")
print("=================================================================")
print(
    f'Test Loss: {mean_loss} \n'
    f'Test Accuracy: {acc} \n'
    f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
    f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
    f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
    f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
    f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
    f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
    f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
    f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
    f'{round(np.mean(res_list==label_list)*100,2)}\n'
)
print("=================================================================")
print("")
print(gtimer)
model_epoch_list.append(last_save)
acc_epoch_list.append(acc)
loss_epoch_list.append(mean_loss)

test step - 4600/4647        
Test Loss: 0.6695421934127808 
Test Accuracy: 74.5211964708414 
TP / FN / ACC: 1205, 1118, 51.87
FP / TN / ACC: 66, 2258, 97.16
PACC / NACC / TACC: 94.81, 66.88, 74.52


inference: 	121328.0 ms/4647 = 26.109 ms (10.109/50.808)



### new data train result - panda
* old data
```
Test Loss: 0.6866675019264221 
Test Accuracy: 76.2 
TP / FN / ACC: 2047, 878, 69.98
FP / TN / ACC: 312, 1763, 84.96
PACC / NACC / TACC: 86.77, 66.76, 76.2
```
* new data
```
Test Loss: 0.06928928196430206 
Test Accuracy: 96.78 
TP / FN / ACC: 151, 30, 83.43
FP / TN / ACC: 131, 4688, 97.28
PACC / NACC / TACC: 53.55, 99.36, 96.78
```
* balanced data
```
Test Loss: 0.5836045145988464 
Test Accuracy: 81.10000000000001 
TP / FN / ACC: 2047, 881, 69.91
FP / TN / ACC: 64, 2008, 96.91
PACC / NACC / TACC: 96.97, 69.51, 81.1
```

### new data train result - indy
* old data
```
Test Loss: 0.7262680530548096 
Test Accuracy: 71.82 
TP / FN / ACC: 1205, 1118, 51.87
FP / TN / ACC: 291, 2386, 89.13
PACC / NACC / TACC: 80.55, 68.09, 71.82
```
* new data
```
Test Loss: 0.0794977992773056 
Test Accuracy: 96.06 
TP / FN / ACC: 129, 64, 66.84
FP / TN / ACC: 133, 4674, 97.23
PACC / NACC / TACC: 49.24, 98.65, 96.06
```
* balanced data
```
Test Loss: 0.6695421934127808 
Test Accuracy: 74.5211964708414 
TP / FN / ACC: 1205, 1118, 51.87
FP / TN / ACC: 66, 2258, 97.16
PACC / NACC / TACC: 94.81, 66.88, 74.52
```

## Test convergence

In [ ]:
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
logpath = os.path.join(ROBOT_MODEL_ROOT, last_model)

model_epoch_list = []
acc_epoch_list = []
loss_epoch_list = []
# last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")])[-1]
# last_save = 'model_1'
for last_save in sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:])):
    model_log_dir = os.path.join(logpath, last_save)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    
    BATCH_SIZE = 50
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:5000,1]
    label_list = np.array(label_list)[:5000,1]
    loss_list = np.array(loss_list)[:5000]
    
    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)
    model_epoch_list.append(last_save)
    acc_epoch_list.append(acc)
    loss_epoch_list.append(mean_loss)

In [ ]:
import matplotlib.pyplot as plt
print(model_epoch_list)
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(acc_epoch_list)
plt.subplot(1,2,2)
plt.plot(loss_epoch_list)

In [ ]:
save_json(ROBOT_TYPE_NAME+".json", {"epoch": np.array(model_epoch_list), "acc": np.array(acc_epoch_list), "loss": np.array(loss_epoch_list)})

## test on shared array

In [ ]:
import SharedArray as sa
BATCH_SIZE = 1
SERVER_PERIOD = 1e-2
# Create an array in shared memory.
robot_type_p = sa.attach("shm://robot_type")
grasp_img_p = sa.attach("shm://grasp_img")
arm_img_p = sa.attach("shm://arm_img")
rh_mask_p = sa.attach("shm://rh_mask")
result_p = sa.attach("shm://result")
query_in = sa.attach("shm://query_in")
response_out = sa.attach("shm://response_out")
query_quit = sa.attach("shm://query_quit")

def query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch):
    grasp_img_p[:] = grasp_img_batch[:]
    arm_img_p[:] = arm_img_batch[:]
    rh_mask_p[:] = rh_mask_batch[:]
    query_in[0] = True
    while not response_out[0]:
        time.sleep(SERVER_PERIOD)
    response_out[0] = False
    return np.copy(result_p)

def quit_shared_server():
    query_quit[0] = True

In [ ]:
LOG_STEP = 100
N_test = len(data_pairs_test)

i_step = 0
data_batch, label_batch = [], []
result_list = []
label_list = []
for data_pair in data_pairs_test:
    i_step += 1
    grasp_img, arm_img, rh_mask, label = load_data(data_pair)
    data_batch.append([grasp_img, arm_img, rh_mask])
    label_batch.append(label)
    if i_step%BATCH_SIZE==0:
        grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
        arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
        rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
        label_batch = np.array(label_batch, dtype=np.int)
        result = query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch)
        result_list.append(result)
        label_list.append(label_batch)
        data_batch, label_batch = [], []
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

In [ ]:
result_all = np.array(result_list)
label_all = np.array(label_list)

In [ ]:
np.mean(np.round(result_all).astype(np.int) == label_all.astype(np.int), axis=0)

In [ ]:
quit_shared_server()

## save gridded data

In [ ]:
data_pair = data_pairs_test[0]
grasp_data = load_pickle(data_pair[0])
arm_data = load_pickle(data_pair[1])
grasp_tar_idx = grasp_data[b'tar']
grasp_tool_idx = grasp_data[b'tool']
arm_tar_idx = arm_data[b'tar']
Tee = grasp_data[b'T_end_effector']
Tej = grasp_data[b'T_end_joint']
Tref_base = grasp_data[b'Tref_base']
reach_lb = grasp_data[b'reach']
retrieve_lb = grasp_data[b'retrieve']
r, th, h = cart2cyl(*Tee[:3,3])
r_ej, th, h_ej = cart2cyl(*Tej[:3,3])
r_class = div_r(r_ej)
h_class = div_h(h_ej)
r_mask = np.zeros(RH_MASK_SIZE)
r_mask[r_class*RH_MASK_STEP:r_class*RH_MASK_STEP+RH_MASK_STEP] = 1
h_mask = np.zeros(RH_MASK_SIZE)
h_mask[h_class*RH_MASK_STEP:h_class*RH_MASK_STEP+RH_MASK_STEP] = 1
rh_mask = np.concatenate([r_mask, h_mask])
# r_ej_list.append(r_ej)
# h_ej_list.append(h_ej)
# reach_lb_list.append(reach_lb)
Tref = SE3(Rot_axis(3, th), Tee[:3,3])
grasp_tool_img = np.zeros(GRASP_SHAPE)
grasp_tar_img = np.zeros(GRASP_SHAPE)
grasp_tool_img[np.unravel_index(grasp_tool_idx, shape=GRASP_SHAPE)] = 1
grasp_tar_img[np.unravel_index(grasp_tar_idx, shape=GRASP_SHAPE)] = 1
arm_img = np.zeros(ARM_SHAPE+(1,))
arm_img[np.unravel_index(arm_tar_idx, shape=ARM_SHAPE)] = 1
grasp_img = np.stack([grasp_tool_img, grasp_tar_img], axis=-1)

In [ ]:
# save_json("tar_arm.json", np.array(np.unravel_index(arm_tar_idx, shape=ARM_SHAPE)).transpose())

In [ ]:
# save_json("tool_effector.json", np.array(np.unravel_index(grasp_tool_idx, shape=GRASP_SHAPE)).transpose())
# save_json("tar_effector.json", np.array(np.unravel_index(grasp_tar_idx, shape=GRASP_SHAPE)).transpose())

## visualize r, h class distribution

In [ ]:
import matplotlib.pyplot as plt
r_ej_list = []
h_ej_list = []
for data_pair in data_pairs_train:
    grasp_data = load_pickle(data_pair[0])
    arm_data = load_pickle(data_pair[1])
    grasp_obj_idx = grasp_data[b'obj']
    grasp_tar_idx = grasp_data[b'tar']
    grasp_tool_idx = grasp_data[b'tool']
    arm_tar_idx = arm_data[b'tar']
    Tee = grasp_data[b'T_end_effector']
    Tej = grasp_data[b'T_end_joint']
    Tref_base = grasp_data[b'Tref_base']
    reach_lb = grasp_data[b'reach']
    retrieve_lb = grasp_data[b'retrieve']
    r, th, h = cart2cyl(*Tee[:3,3])
    r_ej, th, h_ej = cart2cyl(*Tej[:3,3])
    r_ej_list.append(r_ej)
    h_ej_list.append(h_ej)

In [ ]:
def div_r(r):
    return floor(sigmoid((r)/0.1-7)*8)

def div_h(h):
    return floor(sigmoid((h+0.6)/0.2-5)*8)

In [ ]:
plt.subplot(1,2,1)
plt.plot(sorted([div_r(r_ej_) for r_ej_ in r_ej_list]), '.')
plt.subplot(1,2,2)
plt.plot(sorted([div_h(h_ej_) for h_ej_ in h_ej_list]), '.')

In [ ]:
plt.subplot(1,2,1)
plt.plot(sorted([div_r(r_ej_) for r_ej_ in r_ej_list]), '.')
plt.subplot(1,2,2)
plt.plot(sorted([div_h(h_ej_) for h_ej_ in h_ej_list]), '.')

In [ ]:

plt.plot(sorted(np.floor(np.divide(r_ej_list,0.1))), '.')
plt.plot(sorted(np.floor(np.divide(h_ej_list,0.1))), '.')

In [ ]:

plt.plot(sorted(r_ej_list), '.')
plt.plot(sorted(h_ej_list), '.')

In [ ]:
np.max(r_ej_list)

In [ ]:
1.0 - 0.2

In [ ]:
np.min((np.array(r_ej_list)-0.2))

In [ ]:
np.min(np.floor((np.array(r_ej_list)-0.2)/0.05).astype(np.int))

In [ ]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [ ]:
x = div_r_gaussian(sorted(r_ej_list)[5000])

In [ ]:
y = div_h_gaussian(sorted(h_ej_list)[5000])

In [ ]:
plt.figure(figsize=(11,5))
plt.imshow(x[np.newaxis, :], cmap='gray')

In [ ]:
plt.figure(figsize=(16,5))
plt.imshow(y[np.newaxis, :], cmap='gray')

In [ ]:
def div_h_gaussian(h_val):
    return gaussian(h_val, np.arange(-0.5,1.1, 0.05),0.05)

In [ ]:

def div_h(h):
    return floor(sigmoid((h+0.6)/0.2-4.5)*8)

In [ ]:
X = np.repeat(np.transpose([div_r_gaussian(r_val) for r_val in sorted(r_ej_list)]), 100, axis=0)

In [ ]:
X = np.repeat(np.transpose([div_h_gaussian(h_val) for h_val in sorted(h_ej_list)]), 100, axis=0)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(X)

In [ ]:
min(r_ej_list), max(r_ej_list)

In [ ]:
min(h_ej_list), max(h_ej_list)

In [ ]:
plt.plot(sorted(np.clip(np.floor((np.array(r_ej_list)-0.2)/0.05).astype(np.int), 0, 15)))

In [ ]:
Ndat = len(data_pairs_train)
sorted(r_ej_list)